# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Data Preprocessing

In [ ]:
# Read the dataset
df = pd.read_csv('/Users/priyakundu/Documents/NYU Capstone WaterVue Files/Sample_Dataset.csv')
df

## Pivot

In [ ]:
# Pivot the dataframe to get unique values from "Analysis" column as columns
df_pivot = df.pivot_table(index=['Site #', 'Location', 'Sample Date'], columns='Analysis', values='Result').reset_index()

# Group by sample date and location
pivoted_df = df_pivot.groupby(['Sample Date', 'Location']).first().reset_index()

# Print the new dataframe
pivoted_df

In [ ]:
# Renaming the columns using indexing
pivoted_df.columns = [*pivoted_df.columns[:-2], 'Turbidity1', 'Turbidity2']

# Merge the "Turbidity1" and "Turbidity2" columns into a single column named "Turbidity1"
pivoted_df['Turbidity'] = pivoted_df['Turbidity1'].combine_first(pivoted_df['Turbidity2'])

# Drop the "old" columns
pivoted_df.drop(columns=['Turbidity1' , 'Turbidity2'], inplace=True)

# Merge the "Chlorophyll A" and "Chlorophyll a" columns into a single column named "Chlorophyll A"
pivoted_df['Chlorophyll A'] = pivoted_df['Chlorophyll A'].combine_first(pivoted_df['Chlorophyll a'])

# Drop the "Chlorophyll a" column
pivoted_df.drop(columns=['Chlorophyll a'], inplace=True)

# Drop 'Sample Date' column
pivoted_df.drop(['Copper', 'Site #'], axis=1, inplace=True)

# Print the cleaned dataframe
pivoted_df

In [ ]:
# Convert 'Sample Date' column to datetime
pivoted_df['Sample Date'] = pd.to_datetime(pivoted_df['Sample Date'])

# Set 'Sample Date' as index
pivoted_df.set_index('Sample Date', inplace=True)

# Group by 'Location' and resample yearly for each group
resampled_df = pivoted_df.groupby('Location').resample('6M').mean().reset_index()

# Output the resampled data
resampled_df

## Cleaning

In [ ]:
# Find null values in the DataFrame
null_values = resampled_df.isnull()

# Count null values in each column
null_counts = null_values.sum()

print("Null values in each column:")
print(null_counts)

In [ ]:
# Count the number of unique values in the column
num_unique_values = resampled_df['Location'].nunique()

# Count the occurrences of each value in the column
value_counts = resampled_df['Location'].value_counts()

print("Number of unique values:", num_unique_values)
print("Occurrences of each value:")
print(value_counts)

In [ ]:
# Find the mode (most frequent value count)
mode_value_count = value_counts.mode()[0]

mode_value_count

In [ ]:
# Keep only the rows with the mode value count
filtered_df = resampled_df[resampled_df['Location'].map(resampled_df['Location'].value_counts()) == mode_value_count]

filtered_df

In [ ]:
cleaned_df = filtered_df.fillna(method="ffill")

cleaned_df

In [ ]:
# # # Specify the path where you want to save the CSV file
# # csv_file_path = 'Final_Dataframe_WaterQual.csv'

# # Convert DataFrame to CSV
# cleaned_df.to_csv('Ultimate_Dataframe_WaterQual.csv', index=False)

## Splitting

In [ ]:
sorted_df = cleaned_df.sort_values(by='Sample Date', ascending=True)

In [ ]:
# Split data into train and test sets
train_data = sorted_df.iloc[:-4*cleaned_df["Location"].nunique()].sort_values(by=['Location', 'Sample Date'], ascending=True)  # Use all but the last 2 years for training
test_data = sorted_df.iloc[-4*cleaned_df["Location"].nunique():].sort_values(by=['Location', 'Sample Date'], ascending=True)   # Use the last 2 years for testing

# Forecasting

## Chlorophyll A

In [ ]:
param = 'Chlorophyll A'

In [ ]:
predictions = []
for location in train_data["Location"].unique():
    train_df = train_data[train_data["Location"]==location]
    test_df = test_data[test_data["Location"]==location]

    # Fit SARIMA model
    order = (5, 1, 1)       # Example (p, d, q)
    seasonal_order = (1, 2, 0, 2)  # Example (P, D, Q, s) - Changed seasonal order to avoid overlapping AR lags
    model = SARIMAX(train_df[param], order=order, seasonal_order=seasonal_order)
    result = model.fit()

    # Forecast future values
    forecast = result.get_forecast(steps=3)  # Forecasting next 3 years (6 biannual periods) into the future
    forecast_index = pd.date_range(start=train_df["Sample Date"].iloc[-1], periods=4, freq='6M')[1:]
    forecast_values = forecast.predicted_mean

    predictions.append(pd.DataFrame({"Location": [location]*3, "Sample Date": forecast_index,param: forecast_values}))
pred_data = pd.concat(predictions)

In [ ]:
# Concatenate train and test data for plotting
combined_data = pd.concat([train_data.assign(dataset='Train'), test_data.assign(dataset='Test'), pred_data.assign(dataset='Prediction')])

In [ ]:
# Convert "Sample Date" to datetime
combined_data["Sample Date"] = pd.to_datetime(combined_data["Sample Date"])

# Create facet plot
plt.figure(figsize=(20, 15))
sns.set_style("whitegrid")
g = sns.FacetGrid(combined_data, col="Location", hue="dataset", col_wrap=5, height=3, aspect=1.5, sharey=False)

# Iterate over each parameter and map them onto the facet grid for both train and test data
line = g.map(sns.lineplot, "Sample Date", param, marker="o")

# Rotate x-axis labels for better readability
for ax in g.axes.ravel():
    ax.tick_params(labelrotation=45)

plt.subplots_adjust(top=0.95)
g.fig.suptitle('Parameters Over Time for Each Location')

# Adjust legend
plt.legend(loc='upper left')

plt.show()

In [ ]:
maes, mses, rmses = [], [], []
for location in train_data["Location"].unique():
    test_df = test_data[test_data["Location"]==location]
    pred_df = pred_data[pred_data["Location"]==location]
    mae = mean_absolute_error(test_df[param], pred_df[param])
    mse = mean_squared_error(test_df[param], pred_df[param])
    rmse = np.sqrt(mse)

    maes.append(mae)
    mses.append(mse)
    rmses.append(rmse)
accuracy = pd.DataFrame({"Location": train_data["Location"].unique(),
              "MAE": maes,
              "MSE": mses,
              "RMSE": rmses,
            })

In [ ]:
accuracy.sort_values(by="RMSE")

## Dissolved Oxygen

In [ ]:
param = 'Dissolved Oxygen'

In [ ]:
predictions = []
for location in train_data["Location"].unique():
    print("\n\n")
    print(location)
    train_df = train_data[train_data["Location"]==location]
    test_df = test_data[test_data["Location"]==location]

    # Fit SARIMA model
    order = (5, 1, 1)       # Example (p, d, q)
    seasonal_order = (1, 1, 1, 2)  # Example (P, D, Q, s)
    model = SARIMAX(train_df[param], order=order, seasonal_order=seasonal_order)
    result = model.fit()

    # Forecast future values
    forecast = result.get_forecast(steps=3)  # Forecasting next 3 years (6 biannual periods) into the future
    forecast_index = pd.date_range(start=train_df["Sample Date"].iloc[-1], periods=4, freq='6M')[1:]
    forecast_values = forecast.predicted_mean

    predictions.append(pd.DataFrame({"Location": [location]*3, "Sample Date": forecast_index,param: forecast_values}))
pred_data = pd.concat(predictions)

In [ ]:
# Concatenate train and test data for plotting
combined_data = pd.concat([train_data.assign(dataset='Train'), test_data.assign(dataset='Test'), pred_data.assign(dataset='Prediction')])

In [ ]:
# Convert "Sample Date" to datetime
combined_data["Sample Date"] = pd.to_datetime(combined_data["Sample Date"])

# Create facet plot
plt.figure(figsize=(20, 15))
sns.set_style("whitegrid")
g = sns.FacetGrid(combined_data, col="Location", hue="dataset", col_wrap=5, height=3, aspect=1.5, sharey=False)

# Iterate over each parameter and map them onto the facet grid for both train and test data
line = g.map(sns.lineplot, "Sample Date", param, marker="o")

# Rotate x-axis labels for better readability
for ax in g.axes.ravel():
    ax.tick_params(labelrotation=45)

plt.subplots_adjust(top=0.95)
g.fig.suptitle('Parameters Over Time for Each Location')

# Adjust legend
plt.legend(loc='upper left')

plt.show()

In [ ]:
maes, mses, rmses = [], [], []
for location in train_data["Location"].unique():
    test_df = test_data[test_data["Location"]==location]
    pred_df = pred_data[pred_data["Location"]==location]
    mae = mean_absolute_error(test_df[param], pred_df[param])
    mse = mean_squared_error(test_df[param], pred_df[param])
    rmse = np.sqrt(mse)

    maes.append(mae)
    mses.append(mse)
    rmses.append(rmse)
accuracy = pd.DataFrame({"Location": train_data["Location"].unique(),
              "MAE": maes,
              "MSE": mses,
              "RMSE": rmses,
            })

In [ ]:
accuracy.sort_values(by="RMSE")

## Salinity

In [ ]:
param = 'Salinity'

In [ ]:
predictions = []
for location in train_data["Location"].unique():
    train_df = train_data[train_data["Location"]==location]
    test_df = test_data[test_data["Location"]==location]

    # Fit SARIMA model
    order = (5, 1, 1)       # Example (p, d, q)
    seasonal_order = (1, 1, 1, 2)  # Example (P, D, Q, s)
    model = SARIMAX(train_df[param], order=order, seasonal_order=seasonal_order)
    result = model.fit()

    # Forecast future values
    forecast = result.get_forecast(steps=3)  # Forecasting next 3 years (6 biannual periods) into the future
    forecast_index = pd.date_range(start=train_df["Sample Date"].iloc[-1], periods=4, freq='6M')[1:]
    forecast_values = forecast.predicted_mean

    predictions.append(pd.DataFrame({"Location": [location]*3, "Sample Date": forecast_index,param: forecast_values}))
pred_data = pd.concat(predictions)

In [ ]:
# Concatenate train and test data for plotting
combined_data = pd.concat([train_data.assign(dataset='Train'), test_data.assign(dataset='Test'), pred_data.assign(dataset='Prediction')])

In [ ]:
# Convert "Sample Date" to datetime
combined_data["Sample Date"] = pd.to_datetime(combined_data["Sample Date"])

# Create facet plot
plt.figure(figsize=(20, 15))
sns.set_style("whitegrid")
g = sns.FacetGrid(combined_data, col="Location", hue="dataset", col_wrap=5, height=3, aspect=1.5, sharey=False)

# Iterate over each parameter and map them onto the facet grid for both train and test data
line = g.map(sns.lineplot, "Sample Date", param, marker="o")

# Rotate x-axis labels for better readability
for ax in g.axes.ravel():
    ax.tick_params(labelrotation=45)

plt.subplots_adjust(top=0.95)
g.fig.suptitle('Parameters Over Time for Each Location')

# Adjust legend
plt.legend(loc='upper left')

plt.show()

In [ ]:
maes, mses, rmses = [], [], []
for location in train_data["Location"].unique():
    test_df = test_data[test_data["Location"]==location]
    pred_df = pred_data[pred_data["Location"]==location]
    mae = mean_absolute_error(test_df[param], pred_df[param])
    mse = mean_squared_error(test_df[param], pred_df[param])
    rmse = np.sqrt(mse)

    maes.append(mae)
    mses.append(mse)
    rmses.append(rmse)
accuracy = pd.DataFrame({"Location": train_data["Location"].unique(),
              "MAE": maes,
              "MSE": mses,
              "RMSE": rmses,
            })

In [ ]:
accuracy.sort_values(by="RMSE")

## Specific Conductance

In [ ]:
param = 'Specific Conductance'

In [ ]:
predictions = []
for location in train_data["Location"].unique():
    train_df = train_data[train_data["Location"]==location]
    test_df = test_data[test_data["Location"]==location]

    # Fit SARIMA model
    order = (5, 1, 1)       # Example (p, d, q)
    seasonal_order = (1, 1, 1, 2)  # Example (P, D, Q, s)
    model = SARIMAX(train_df[param], order=order, seasonal_order=seasonal_order)
    result = model.fit()

    # Forecast future values
    forecast = result.get_forecast(steps=3)  # Forecasting next 3 years (6 biannual periods) into the future
    forecast_index = pd.date_range(start=train_df["Sample Date"].iloc[-1], periods=4, freq='6M')[1:]
    forecast_values = forecast.predicted_mean

    predictions.append(pd.DataFrame({"Location": [location]*3, "Sample Date": forecast_index,param: forecast_values}))
pred_data = pd.concat(predictions)

In [ ]:
# Concatenate train and test data for plotting
combined_data = pd.concat([train_data.assign(dataset='Train'), test_data.assign(dataset='Test'), pred_data.assign(dataset='Prediction')])

In [ ]:
# Convert "Sample Date" to datetime
combined_data["Sample Date"] = pd.to_datetime(combined_data["Sample Date"])

# Create facet plot
plt.figure(figsize=(20, 15))
sns.set_style("whitegrid")
g = sns.FacetGrid(combined_data, col="Location", hue="dataset", col_wrap=5, height=3, aspect=1.5, sharey=False)

# Iterate over each parameter and map them onto the facet grid for both train and test data
line = g.map(sns.lineplot, "Sample Date", param, marker="o")

# Rotate x-axis labels for better readability
for ax in g.axes.ravel():
    ax.tick_params(labelrotation=45)

plt.subplots_adjust(top=0.95)
g.fig.suptitle('Parameters Over Time for Each Location')

# Adjust legend
plt.legend(loc='upper left')

plt.show()

In [ ]:
maes, mses, rmses = [], [], []
for location in train_data["Location"].unique():
    test_df = test_data[test_data["Location"]==location]
    pred_df = pred_data[pred_data["Location"]==location]
    mae = mean_absolute_error(test_df[param], pred_df[param])
    mse = mean_squared_error(test_df[param], pred_df[param])
    rmse = np.sqrt(mse)

    maes.append(mae)
    mses.append(mse)
    rmses.append(rmse)
accuracy = pd.DataFrame({"Location": train_data["Location"].unique(),
              "MAE": maes,
              "MSE": mses,
              "RMSE": rmses,
            })

In [ ]:
accuracy.sort_values(by="RMSE")

## Total Nitrogen

In [ ]:
param = 'Total Nitrogen'

In [ ]:
predictions = []
for location in train_data["Location"].unique():
    train_df = train_data[train_data["Location"]==location]
    test_df = test_data[test_data["Location"]==location]

    # Fit SARIMA model
    order = (5, 1, 1)       # Example (p, d, q)
    seasonal_order = (1, 1, 1, 2)  # Example (P, D, Q, s)
    model = SARIMAX(train_df[param], order=order, seasonal_order=seasonal_order)
    result = model.fit()

    # Forecast future values
    forecast = result.get_forecast(steps=3)  # Forecasting next 3 years (6 biannual periods) into the future
    forecast_index = pd.date_range(start=train_df["Sample Date"].iloc[-1], periods=4, freq='6M')[1:]
    forecast_values = forecast.predicted_mean

    predictions.append(pd.DataFrame({"Location": [location]*3, "Sample Date": forecast_index,param: forecast_values}))
pred_data = pd.concat(predictions)

In [ ]:
# Concatenate train and test data for plotting
combined_data = pd.concat([train_data.assign(dataset='Train'), test_data.assign(dataset='Test'), pred_data.assign(dataset='Prediction')])

In [ ]:
# Convert "Sample Date" to datetime
combined_data["Sample Date"] = pd.to_datetime(combined_data["Sample Date"])

# Create facet plot
plt.figure(figsize=(20, 15))
sns.set_style("whitegrid")
g = sns.FacetGrid(combined_data, col="Location", hue="dataset", col_wrap=5, height=3, aspect=1.5, sharey=False)

# Iterate over each parameter and map them onto the facet grid for both train and test data
line = g.map(sns.lineplot, "Sample Date", param, marker="o")

# Rotate x-axis labels for better readability
for ax in g.axes.ravel():
    ax.tick_params(labelrotation=45)

plt.subplots_adjust(top=0.95)
g.fig.suptitle('Parameters Over Time for Each Location')

# Adjust legend
plt.legend(loc='upper left')

plt.show()

In [ ]:
maes, mses, rmses = [], [], []
for location in train_data["Location"].unique():
    test_df = test_data[test_data["Location"]==location]
    pred_df = pred_data[pred_data["Location"]==location]
    mae = mean_absolute_error(test_df[param], pred_df[param])
    mse = mean_squared_error(test_df[param], pred_df[param])
    rmse = np.sqrt(mse)

    maes.append(mae)
    mses.append(mse)
    rmses.append(rmse)
accuracy = pd.DataFrame({"Location": train_data["Location"].unique(),
              "MAE": maes,
              "MSE": mses,
              "RMSE": rmses,
            })

In [ ]:
accuracy.sort_values(by="RMSE")

## Total Phosphorus

In [ ]:
param = 'Total Phosphorus'

In [ ]:
predictions = []
for location in train_data["Location"].unique():
    train_df = train_data[train_data["Location"]==location]
    test_df = test_data[test_data["Location"]==location]

    # Fit SARIMA model
    order = (5, 1, 1)       # Example (p, d, q)
    seasonal_order = (1, 1, 1, 2)  # Example (P, D, Q, s)
    model = SARIMAX(train_df[param], order=order, seasonal_order=seasonal_order)
    result = model.fit()

    # Forecast future values
    forecast = result.get_forecast(steps=3)  # Forecasting next 3 years (6 biannual periods) into the future
    forecast_index = pd.date_range(start=train_df["Sample Date"].iloc[-1], periods=4, freq='6M')[1:]
    forecast_values = forecast.predicted_mean

    predictions.append(pd.DataFrame({"Location": [location]*3, "Sample Date": forecast_index,param: forecast_values}))
pred_data = pd.concat(predictions)

In [ ]:
# Concatenate train and test data for plotting
combined_data = pd.concat([train_data.assign(dataset='Train'), test_data.assign(dataset='Test'), pred_data.assign(dataset='Prediction')])

In [ ]:
# Convert "Sample Date" to datetime
combined_data["Sample Date"] = pd.to_datetime(combined_data["Sample Date"])

# Create facet plot
plt.figure(figsize=(20, 15))
sns.set_style("whitegrid")
g = sns.FacetGrid(combined_data, col="Location", hue="dataset", col_wrap=5, height=3, aspect=1.5, sharey=False)

# Iterate over each parameter and map them onto the facet grid for both train and test data
line = g.map(sns.lineplot, "Sample Date", param, marker="o")

# Rotate x-axis labels for better readability
for ax in g.axes.ravel():
    ax.tick_params(labelrotation=45)

plt.subplots_adjust(top=0.95)
g.fig.suptitle('Parameters Over Time for Each Location')

# Adjust legend
plt.legend(loc='upper left')

plt.show()

In [ ]:
maes, mses, rmses = [], [], []
for location in train_data["Location"].unique():
    test_df = test_data[test_data["Location"]==location]
    pred_df = pred_data[pred_data["Location"]==location]
    mae = mean_absolute_error(test_df[param], pred_df[param])
    mse = mean_squared_error(test_df[param], pred_df[param])
    rmse = np.sqrt(mse)

    maes.append(mae)
    mses.append(mse)
    rmses.append(rmse)
accuracy = pd.DataFrame({"Location": train_data["Location"].unique(),
              "MAE": maes,
              "MSE": mses,
              "RMSE": rmses,
            })

In [ ]:
accuracy.sort_values(by="RMSE")

## Turbidity

In [ ]:
param = 'Turbidity'

In [ ]:
predictions = []
for location in train_data["Location"].unique():
    train_df = train_data[train_data["Location"]==location]
    test_df = test_data[test_data["Location"]==location]

    # Fit SARIMA model
    order = (5, 1, 1)       # Example (p, d, q)
    seasonal_order = (1, 1, 1, 2)  # Example (P, D, Q, s)
    model = SARIMAX(train_df[param], order=order, seasonal_order=seasonal_order)
    result = model.fit()

    # Forecast future values
    forecast = result.get_forecast(steps=3)  # Forecasting next 3 years (6 biannual periods) into the future
    forecast_index = pd.date_range(start=train_df["Sample Date"].iloc[-1], periods=4, freq='6M')[1:]
    forecast_values = forecast.predicted_mean

    predictions.append(pd.DataFrame({"Location": [location]*3, "Sample Date": forecast_index,param: forecast_values}))
pred_data = pd.concat(predictions)

In [ ]:
# Concatenate train and test data for plotting
combined_data = pd.concat([train_data.assign(dataset='Train'), test_data.assign(dataset='Test'), pred_data.assign(dataset='Prediction')])

In [ ]:
# Convert "Sample Date" to datetime
combined_data["Sample Date"] = pd.to_datetime(combined_data["Sample Date"])

# Create facet plot
plt.figure(figsize=(20, 15))
sns.set_style("whitegrid")
g = sns.FacetGrid(combined_data, col="Location", hue="dataset", col_wrap=5, height=3, aspect=1.5, sharey=False)

# Iterate over each parameter and map them onto the facet grid for both train and test data
line = g.map(sns.lineplot, "Sample Date", param, marker="o")

# Rotate x-axis labels for better readability
for ax in g.axes.ravel():
    ax.tick_params(labelrotation=45)

plt.subplots_adjust(top=0.95)
g.fig.suptitle('Parameters Over Time for Each Location')

# Adjust legend
plt.legend(loc='upper left')

plt.show()

In [ ]:
maes, mses, rmses = [], [], []
for location in train_data["Location"].unique():
    test_df = test_data[test_data["Location"]==location]
    pred_df = pred_data[pred_data["Location"]==location]
    mae = mean_absolute_error(test_df[param], pred_df[param])
    mse = mean_squared_error(test_df[param], pred_df[param])
    rmse = np.sqrt(mse)

    maes.append(mae)
    mses.append(mse)
    rmses.append(rmse)
accuracy = pd.DataFrame({"Location": train_data["Location"].unique(),
              "MAE": maes,
              "MSE": mses,
              "RMSE": rmses,
            })

In [ ]:
accuracy.sort_values(by="RMSE")